# 베르누이 나이브 베이즈 분류

In [2]:
import numpy as np
import pandas as pd

# 베르누이 나이브베이즈를 위한 라이브러리를 임포트합니다
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

# 모델의 정확도 평가를 위해 임포트합니다
from sklearn.metrics import accuracy_score

# 문제 정의
이메일 제목과 레이블 데이터를 활용해 베르누이 나이브 베이즈 분류로 스팸 메일을 분류한다.

In [3]:
email_list = [
                {'email title': 'free game only today', 'spam': True},
                {'email title': 'cheapest flight deal', 'spam': True},
                {'email title': 'limited time offer only today only today', 'spam': True},
                {'email title': 'today meeting schedule', 'spam': False},
                {'email title': 'your flight schedule attached', 'spam': False},
                {'email title': 'your credit card statement', 'spam': False}
             ]
df = pd.DataFrame(email_list)
df

email title   spam
0                      free game only today   True
1                      cheapest flight deal   True
2  limited time offer only today only today   True
3                    today meeting schedule  False
4             your flight schedule attached  False
5                your credit card statement  False

### 데이터 다듬기

사이킷런의 베르누이 나이브 베이즈 분류기(BernoulliNB)는 숫자만 다루기 때문에 True와 False를 1과 0으로 치환한다.

In [4]:
df['label'] = df['spam'].map({True:1, False:0})
df

email title   spam  label
0                      free game only today   True      1
1                      cheapest flight deal   True      1
2  limited time offer only today only today   True      1
3                    today meeting schedule  False      0
4             your flight schedule attached  False      0
5                your credit card statement  False      0

In [5]:
# 학습에 사용될 데이터와 분류값을 나눕니다
df_x = df['email title']
df_y = df['label']

베르누이 나이브 베이즈의 입력 데이터는 고정된 크기의 벡터여야 한다.    
사이킷런의 CountVectorizer를 이용하면 간편하게 특정 데이터 안의 모든 단어를 포함한 고정 길이 벡터를 만들 수 있다.  
binary=True를 파라미터를 넘겨줌으로써, 각각의 이메일마다 단어가 한번 이상 출현하면 1, 출현하지 않을 경우 0으로 표시하게 한다.

In [6]:
cv = CountVectorizer(binary=True)
x_traincv = cv.fit_transform(df_x)

In [7]:
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]])

위의 행렬에서 볼 수 있듯, 데이터에서 총 17개의 단어가 발견되어, 각각의 이메일이 17개의 크기를 갖는 벡터로 표현되었다.  
또한 베르누이 나이브베이즈에 사용하기 위해 중복된 단어가 이메일 제목에 있더라도, 중복된 횟수로 표현하는 것이 아니라 1로 표현된 것을 확인할 수 있다.  

아래 이메일 제목은 
limited time offer only today only today: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]

고정된 크기의 벡터에 어떠한 단어들이 포함되어 있는지 확인할 수도 있다. (inverse_transform 사용)

In [8]:
cv.inverse_transform(encoded_input)

[array(['free', 'game', 'only', 'today'], dtype='<U9'),
 array(['cheapest', 'deal', 'flight'], dtype='<U9'),
 array(['limited', 'offer', 'only', 'time', 'today'], dtype='<U9'),
 array(['meeting', 'schedule', 'today'], dtype='<U9'),
 array(['attached', 'flight', 'schedule', 'your'], dtype='<U9'),
 array(['card', 'credit', 'statement', 'your'], dtype='<U9')]

고정된 벡터의 각 인덱스가 어떤 단어를 의미하는지 궁금하다면 get_feature_names()를 이용한다.

In [9]:
cv.get_feature_names()

['attached',
 'card',
 'cheapest',
 'credit',
 'deal',
 'flight',
 'free',
 'game',
 'limited',
 'meeting',
 'offer',
 'only',
 'schedule',
 'statement',
 'time',
 'today',
 'your']

# 베르누이 나이브 베이즈 모델 학습

#### 스무딩(smooting)
이산적인 데이터의 경우 빈도수가 0인 경우가 발생한다. 예를 들어 나이브 베이즈 기반 스팸 메일 필터를 가동시키는데,
학습 데이터에 없던 단어가 실제 상황에서 나타나게 되면 확률이 0이 되어 스팸 분류가 어려워진다.
이런 문제를 극복하기 위해 나온 기술이 스무딩(smoothing)이다.
스무딩은 학습 데이터에 없던 데이터가 출현해도 빈도수에 1을 더해서 확률이 0이 되는 현상을 방지한다.

사이킷런의 베르누이 나이브 베이즈 분류기는 기본적으로 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 잘 진행된다.

In [10]:
bnb = BernoulliNB()
y_train = df_y.astype('int')
bnb.fit(x_traincv, y_train)

BernoulliNB()

In [11]:
# 테스트 데이터 다듬기
test_email_list = [
                {'email title': 'free flight offer', 'spam': True},
                {'email title': 'hey traveler free flight deal', 'spam': True},
                {'email title': 'limited free game offer', 'spam': True},
                {'email title': 'today flight schedule', 'spam': False},
                {'email title': 'your credit card attached', 'spam': False},
                {'email title': 'free credit card offer only today', 'spam': False}
             ]
test_df = pd.DataFrame(test_email_list)
test_df['label'] = test_df['spam'].map({True:1,False:0})
test_x=test_df["email title"]
test_y=test_df["label"]
x_testcv=cv.transform(test_x)

# 테스트

In [12]:
predictions = bnb.predict(x_testcv)

In [13]:
accuracy_score(test_y, predictions)

0.8333333333333334